# Project Oxford: Face API


In [1]:
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter
import httplib, urllib, base64 #General API Usage

In [2]:
# Variables

_url_detect = 'https://api.projectoxford.ai/face/v1.0/detect'
_url_verify = 'https://api.projectoxford.ai/face/v1.0/verify'
_url_group = 'https://api.projectoxford.ai/face/v1.0/group'
_key = '0b8afa8909154432bafeed6a3217fcde' #Here you have to paste your primary key
_maxNumRetries = 10

## 1. Face Detection (To get faceId)

In [3]:
def detectionRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url_detect, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [4]:
def renderResultOnImage( result, img ):
    
    """Display the obtained results onto the input image"""

    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        cv2.rectangle( img,(faceRectangle['left'],faceRectangle['top']),
                           (faceRectangle['left']+faceRectangle['width'], faceRectangle['top'] + faceRectangle['height']),
                       color = (255,0,0), thickness = 1 )

        faceLandmarks = currFace['faceLandmarks']

        for _, currLandmark in faceLandmarks.items():
            cv2.circle( img, (int(currLandmark['x']),int(currLandmark['y'])), color = (0,255,0), thickness= -1, radius = 1 )

    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        faceAttributes = currFace['faceAttributes']

        textToWrite = "%c (%d)" % ( 'M' if faceAttributes['gender']=='male' else 'F', faceAttributes['age'] )
        cv2.putText( img, textToWrite, (faceRectangle['left'],faceRectangle['top']-15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1 )

##   Face detection

폴더 내에 있는 페이스 아이디 저장하기: test_faceId

In [23]:
import os

# Face detection parameters
params = { 'returnFaceAttributes': 'age,gender', 
           'returnFaceLandmarks': 'true'} 

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None

cur_dir = '/Users/robot/git_ryan/MS_Face_API/image/Park/park_etc/'
test_file = os.listdir(cur_dir)

test_faceId = [] #face id info
file_name = [] #directory file name
data_name = [] #data directory

for i in xrange(0, len(test_file)):
    file_name.append(cur_dir + test_file[i])
    
    with open (file_name[i], 'rb') as f:
        data = f.read()
        data_name.append(data)

    try:
        result = detectionRequest( json, data, headers, params )
        test_faceId.append( result[0]['faceId'])
        time.sleep(1)
        print("success")
    except:
        print("fail filename: %s" %file_name[i])

success
success
success
success
success
success


In [ ]:
len(test_file)
len(test_faceId)

In [28]:
test_faceId[0]

u'c8091e09-490e-4209-908e-6008594f5a89'

6

6

In [16]:
import cPickle as pickle

pickle.dump( test_faceId, open( "faceId.p", "wb"))

face_id = pickle.load( open ( "faceId.p", "rb"))
print(face_id)

[u'06f10019-6190-43c9-9a50-985592cb6229', u'41e25daa-7ad4-4e56-be06-919dae46c2dc', u'e3c7af6b-74a4-4401-9e44-87a3fc7d3f5f', u'ae5012d4-f2ea-4aa1-8302-af8f8432f820', u'e751ccc5-3ad4-49fa-885d-c54ebc1827d3', u'c9042298-cf29-4705-8726-fb2eb9390d1b', u'b80528a0-5753-496c-8f40-2f66368219a3', u'ca551e36-98af-42c8-b143-5e75fd7ce906', u'0db27b01-cfcd-438d-8241-61f06ec0e27c', u'4ff5f7b7-5c60-4f0a-a08b-99979618aae6', u'68974d35-1ff7-4b41-9822-ac80be175524', u'd65fddd2-e44e-4228-9f08-a0c3e7e57f4a', u'834e2ef4-5c1b-4759-bee2-d9afbf8cce63', u'332ad24b-5bb1-40c6-8d81-8df996e074f9']


u'06f10019-6190-43c9-9a50-985592cb6229'

In [ ]:
try:

(role, line_spoken) = each_line.split(':')

line_spoken = line_spoken.strip()

if role == 'Man':

man.append(line_spoken)

elif role == 'Other Man':

other.append(line_spoken)

else:

pass

except ValueError:

pass

 

data.close()

except IOError:

print('The datafile is missing!')

## Face Data 출력하기

In [ ]:
#TODO 수정필요

if result[0] is not None:
    # Load the original image from disk
    data8uint = np.fromstring( data[0], np.uint8 ) # Convert string to an unsigned int array
    img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )

    renderResultOnImage( result[0], img )

    ig, ax = plt.subplots(figsize=(15, 20))
    ax.imshow( img )

## 2. Face Grouping (Need faceId)

In [8]:
def groupRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url_group, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [9]:

test_faceId

[u'c82aa806-0849-489d-8fa0-6297b7bd6e20',
 u'fc46ab2a-6432-4d4c-9cc0-3b2d648c3de3',
 u'20539f8b-a0ab-43da-ab17-7fd001b20d94',
 u'95294793-6b0d-470e-a18f-136d514fbca1',
 u'4254d400-4ccd-486b-b0ad-7d3eee5f6169',
 u'364d8f3f-64e6-4274-bee5-4ede40d8400f',
 u'a61d5b8f-5e5c-4e0a-9b80-06e2845f42e5',
 u'4c59351c-cd49-4ca6-a3a0-3c5c58c2d9ae',
 u'31f6a1f0-5eea-495a-8a2f-75ad258d4b21',
 u'b0c998aa-cde5-472f-a114-ed9c5a673f3f',
 u'24a2ab77-efe8-4236-b592-ec0762a83804',
 u'6d950d29-015a-4b00-9a80-b8090df66a44',
 u'f57e7f16-567f-4f0d-863d-eaf69dd373f3',
 u'f7b81aca-f3ae-442b-abd6-d0d0261f3932']

In [14]:
params = None

headers = dict()

headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json'

data = None

json = { 'faceIds': test_faceId } 

result = groupRequest( json, data, headers, params )
print(result['groups'])

[[u'4c59351c-cd49-4ca6-a3a0-3c5c58c2d9ae', u'31f6a1f0-5eea-495a-8a2f-75ad258d4b21', u'fc46ab2a-6432-4d4c-9cc0-3b2d648c3de3', u'6d950d29-015a-4b00-9a80-b8090df66a44', u'95294793-6b0d-470e-a18f-136d514fbca1', u'4254d400-4ccd-486b-b0ad-7d3eee5f6169', u'b0c998aa-cde5-472f-a114-ed9c5a673f3f', u'24a2ab77-efe8-4236-b592-ec0762a83804'], [u'20539f8b-a0ab-43da-ab17-7fd001b20d94', u'c82aa806-0849-489d-8fa0-6297b7bd6e20'], [u'a61d5b8f-5e5c-4e0a-9b80-06e2845f42e5', u'f7b81aca-f3ae-442b-abd6-d0d0261f3932'], [u'364d8f3f-64e6-4274-bee5-4ede40d8400f', u'f57e7f16-567f-4f0d-863d-eaf69dd373f3']]


## 3. Face Verification (Need faceId)

In [166]:
def verificationRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url_verify, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [195]:
# Face Simliarity parameters

#params = { 'faceId1': 'a0747ea2-b8a3-44ec-80f0-04b6a8b0f6cc', 
#           'faceId2': '77a2d1d0-c43f-4033-bfc5-c5af55ffe72c'} 

params = None

headers = dict()

headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json'
                                                    
#json = None

data = None

for i in xrange(len(test_file)):
    json = { 'faceId1': test_faceId[i], 'faceId2': test_faceId[i+1] } 
    result = verificationRequest( json, data, headers, params )
    print(result)

#print(result)
#print(result["isIdentical"])
#print(result["confidence"])


#print(result[0]["faceId"])
#a0747ea2-b8a3-44ec-80f0-04b6a8b0f6cc
#77a2d1d0-c43f-4033-bfc5-c5af55ffe72c

KeyboardInterrupt: 